# 네이버 뉴스 기사 스크래핑
>## Crawling Target
1. news_url
2. news_title
3. news_sub_title
4. news_author
5. news_date
6. news_article
7. news_img_path
8. news_source

```
# 코드로 형식 지정됨
```



In [29]:
%%time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd


# 접근정보 추가
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15'}

# 각 기사의 URL 주소를 리스트에 담는 함수
# input 변수 = page_num: 크롤링할 페이지 수, code: 카테고리, date: 날짜
def make_urllist(page_num, code, date): 

  # URL 주소를 저장하기 위한 비어있는 리스트 미리 세팅.
  urllist= []

  # 1 ~ page_num까지 정해진 페이지만큼 반복.
  # <예시> range(1,2) = 1 페이지만 크롤링
  # range(1,11) 은 숫자 1부터 10까지 (1이상 11미만)의 숫자를 데이터로 갖는 객체
  for i in range(1, page_num + 1):

	  # 입력한 변수를 조합하여 생성한 주소를  url 이라는 변수에 저장
    # 이때, 문자열과 숫자는 더할 수 없으므로 숫자를 문자열로 바꿔주어야 함
    url = 'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid='+str(code)+'&date='+str(date)+'&page='+str(i)

	  # requets 패키지의 get 함수로 호출하여 html 정보를 news라는 변수에 저장
	  # import로 호출한 패키지의 함수 사용 방법은 '패키지이름.모듈이름' 따라서 requests.get 으로 사용
    news = requests.get(url, headers=headers)
    news.content

    # BeautifulSoup 모듈을 사용하여 HTML 페이지를 분석
    soup = BeautifulSoup(news.content, 'html.parser')

    # 각 페이지의 뉴스가 news_list 라는 리스트에 저장
    #html 구조 확인
    #페이지당 뉴스기사는 20개이며 상단 10개 뉴스기사는 type06_headline 클래스에 하단 10개 뉴스기사는 type06 클래스에 들어있음. 따라서 크롤링 시 두 클래스의 태그속 url을 모두 가져와야함.
    news_lists = soup.select('.type06_headline li dl')
    news_lists.extend(soup.select('.type06 li dl'))

    # 뉴스 리스트에 있는 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소' 텍스트만 가져와서 list에 append사용해 쌓기
    for news_list in news_lists:
        urllist.append(news_list.a.get('href'))
  return urllist

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 21.2 µs


In [31]:
%%time
#언론사 뉴스 검색
news_urllist = make_urllist(1, "023", 20220221)
# make_urllist(page_num, code, date)
  # page_num: 가지고올 페이지 수(한 페이지당 기사 20개)
  # code: 언론사 코드
    # 조선일보: 023
    # 중앙일보: 025
    # 동아일보: 020
    # 한겨례: 028
    # 매일경제: 009
  # date: YYYYMMDD

print('뉴스 기사의 개수 :',len(news_urllist))
print(news_urllist)

뉴스 기사의 개수 : 20
['https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674012', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674011', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674010', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674009', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674008', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674007', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674006', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674005', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674004', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674003', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674002', 'https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0

In [32]:
%%time
data = []
for news_url in news_urllist:
  json_data = {}
  
  # news_url: 기사 원문 URL
  json_data["news_url"] = news_url
  print(news_url)

  news_url = requests.get(news_url, headers=headers)
  news_url.content
  news_page_html = BeautifulSoup( news_url.content, 'html.parser')

  # news_title: 기사 제목
  news_title = news_page_html.select_one('#articleTitle').text
  json_data["news_title"] = news_title
  print(news_title)

  # news_subtitle: 부제목
  if news_page_html.select_one(".media_end_summary"):
    news_subtitle = news_page_html.select_one(".media_end_summary").text
  else:
    news_subtitle = "null"
  json_data["news_subtitle"] = news_subtitle
  print(news_subtitle)

  # news_author: 기자
  if news_page_html.select_one(".journalistcard_summary_name"):
    news_author = news_page_html.select_one(".journalistcard_summary_name").text
    news_author = re.sub("[기자]","", news_author).strip()
  elif news_page_html.select_one(".b_text"):
    news_author = news_page_html.select_one(".b_text").text
    news_author = re.sub("[|기자|인턴|특파원|\s|\d|a-z|(|@|.|)|]","", news_author).strip()
  else:
    news_author = "null"
  json_data["news_author"] = news_author
  print(news_author)

  # news_date: 날짜
  if news_page_html.select_one(".t11"):
    news_data = news_page_html.select_one(".t11").text
  #   # news_date = str(news_data)
  #   news_date = news_date.replace('<span class="t11">', "")
  #   news_date = news_date.replace("</span>", "")
  else:
    news_data = "null"
  json_data["news_data"] = news_data
  print(news_data)

  # news_article: 기사 내용
  if news_page_html.find('div', {'id':'articleBodyContents'}):
    news_article = news_page_html.find('div', {'id':'articleBodyContents'}).text.strip()
    news_article = news_article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
    news_article = news_article.replace("function _flash_removeCallback() {}", "")
    ignore_article = "|\n|\t|\xa0|[WEEKLY|BIZ]|Biz|Calendar|[앵커]|[리포트]|이미지출처처=연합뉴스스]|[KBS|울산]|[포토]|[연합뉴스]|[경향신문]|"
    news_article = re.sub(ignore_article,"", news_article)
  else:
    news_article = "null"
  json_data["news_article"] = news_article
  print(news_article)

  # news_img_path: 기사 img 경로
  # if news_soup.select("figure"):
  #     news_img_path = news_soup.select_one("figure").img.get("src")
  # else:
  #     news_img_path = "null"
  # json_data["news_img_path"] = news_img_path

  # news_source : 신문사
  if news_page_html.select_one(".c_text"):
    news_source = news_page_html.select_one(".c_text").get_text().strip()
    news_source = news_source.replace("Copyright ⓒ ", "")
    news_source = news_source.replace(". All rights reserved. 무단 전재 및 재배포 금지.", "")
  else:
    news_source = "null"

  json_data["news_source"] = news_source
  print(news_source)

  data.append(json_data)

print(data)
  

https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=023&aid=0003674012
중국·IOC “성공적 올림픽” 자화자찬… WP “스캔들 올림픽으로 기억될 것”
null
송원형
2022.02.21. 오전 12:37
베이징 올림픽 ‘엇갈린 평가’선수들의 땀과 투혼 빛났지만 인권·판정·도핑 논란으로 시끌베이징 동계 올림픽이 20일 폐회식을 끝으로 막을 내렸다. 베이징 올림픽은 작년 여름에 열린 도쿄 하계 올림픽에 이어 코로나 팬데믹 속에서 열린 두 번째 올림픽이다. 다행히 코로나 집단감염이 발생하거나 코로나로 기가 중단되는 일도 없었다. 하지만 중국 내 장 위구르 자치구 인권침해부터 중국의 ‘한복 공정’, 편파 판정, ‘피겨 요정’ 카밀라 발예바(16·러시아올림픽위원회)의 도핑 이슈까지 시작부터 끝까지 논란이 끊이지 않았다. 미국 워싱턴(P)는 “‘캔들 올림픽’으로 기억될 것”이라며 혹평했다.◇정치·판정·도핑 이슈로 시끌중국은 개회식부터 장 위구르 출 크로컨 선수 디니거 이라무장(21)을 성화 마지막 점화자로 내세우면서 전 세계의 허를 찔렀다. 미국 등 서방국가가 중국 내 소수민족 인권을 제 삼으며 ‘외교적 보이콧’을 하자 “올림픽을 정치적으로 이용해선 안 된다”고 했던 중국이 개회식에서 의도적으로 인권 제의 중심에 있던 이라무장을 내세워 오히려 정치적인 입장을 나타냈다는 비난을 받았다.개회식 때 중국 내 소수민족 중 하나인 조선족 대표가 한복을 입고 나온 것을 두고도 국내에선 “화 공정 아니냐”는 비판 여론이 일었다. 황대헌(23)과 이준서(22)가 남자 쇼랙 1000m 준결승에서 석찮은 판정으로 실격당하고, 중국 선수가 반칙으로 보이는 행동을 하고도 금메달을 획득하자 국내 반중 정서는 극에 달했다. 판정 시비 제는 키점프와 노보드에서도 이어졌고, 독일·노르웨이·일본 등 다른 나라들도 강하게 어필했다.피겨 케이팅 여자 싱글의 강력한 우승 후보 발예바의 도핑 논란은 베이징 올림픽을 강타한 태풍이었다. 그

In [33]:
# json 파일로 저장
import json
with open("Naver_News.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file,  ensure_ascii=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')